## 문제1)
*출처 : 롯데멤버스, L.pay|L.POINT, 제6회 L.POINT Big Data Competition*

### 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
%matplotlib inline

In [2]:
cs = pd.read_csv('L사_고객정보.csv')
gd = pd.read_csv('L사_상품정보.csv')
tr = pd.read_csv('L사_거래정보.csv')

gd.pd_c = gd.pd_c.astype(str).str.zfill(4) # 상품과 거래 데이터의 pd_c 컬럼의 데이터 유형이 서로 달라 맞춰줌. 자리수까지도 맞춰줘야 함. 
df = pd.merge(tr, cs).merge(gd, on='pd_c')
df.de_dt = df.de_dt.astype(str).astype('datetime64') # 날짜정보를 담고 있는 de_dt 컬럼의 데이터 유형이 정수이기 때문

df.head()

,clnt_id,trans_id,trans_seq,biz_unit,pd_c,de_dt,de_tm,buy_am,buy_ct,clnt_gender,clnt_age,clac_nm1,clac_nm2,clac_nm3
0,21922,104999,3,A03,0182,2019-09-20,12:41,10900,1,F,50,Chilled Foods,Chilled Instant Foods,Chilled Soups
1,39423,105124,10,A03,0182,2019-09-20,17:26,21800,2,F,50,Chilled Foods,Chilled Instant Foods,Chilled Soups
2,39423,89469,2,A03,0182,2019-09-01,03:32,25800,2,F,50,Chilled Foods,Chilled Instant Foods,Chilled Soups
3,39423,88436,1,A03,0182,2019-08-30,17:15,25800,2,F,50,Chilled Foods,Chilled Instant Foods,Chilled Soups
4,18362,50872,3,A03,0182,2019-07-15,09:46,31600,4,F,40,Chilled Foods,Chilled Instant Foods,Chilled Soups


### ratings_matrx 생성하기
- 행은 clnt_id(고객ID), 열은 clac_nm3(상품 소분류명)로, vales='pd_c'(상품 소분류코드)로 pivoting 수행. 구매가 없는 값(NaN)은 모두 0으로 변환

In [3]:
ratings_matrix = pd.pivot_table(df, index='clnt_id', columns='clac_nm3', values = 'pd_c', 
                                aggfunc=lambda x: 1 if len(x) >= 1 else 0, fill_value=0)

print(ratings_matrix.shape)
ratings_matrix

(11270, 1662)


clac_nm3,,Accessory Bags,Accident Prevention Equipment,Adhesive Tapes,Adhesives,Adult's Bed Covers and Skirts,Adult's Diapers,Adult's Disposable Briefes,Adults' Bed Fillings,Adults' Bedding Sets,...,Yoga / Pilates Clothing,Yoga Mats / Exercise Mats,Yoghurts / Chunggukjang Makers,Yogurt Drinks,Young Pumpkins,Young Radishes,Yuzus / Quinces,Zipper Poly Bags / Plastic Bags,kelp,life Vests / Safety Accessories
clnt_id,,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72373,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
72400,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
72410,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 코사인 유사도 구하기
- cosine_similarity를 사용하여 코사인 유사도를 구하고 데이터프레임 형태로 merchan_sim_df에 저장하세요

In [4]:
# ratings_matrix의 행과 열의 위치를 변경
ratings_matrix_T = ratings_matrix.transpose()

In [5]:
from sklearn.metrics.pairwise import cosine_similarity

merchan_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)

merchan_sim_df = pd.DataFrame(data=merchan_sim, index=ratings_matrix.columns,
                          columns=ratings_matrix.columns)
print(merchan_sim_df.shape)
merchan_sim_df.head(3)

(1662, 1662)


clac_nm3,,Accessory Bags,Accident Prevention Equipment,Adhesive Tapes,Adhesives,Adult's Bed Covers and Skirts,Adult's Diapers,Adult's Disposable Briefes,Adults' Bed Fillings,Adults' Bedding Sets,...,Yoga / Pilates Clothing,Yoga Mats / Exercise Mats,Yoghurts / Chunggukjang Makers,Yogurt Drinks,Young Pumpkins,Young Radishes,Yuzus / Quinces,Zipper Poly Bags / Plastic Bags,kelp,life Vests / Safety Accessories
clac_nm3,,,,,,,,,,,,,,,,,,,,,
,1.000000,0.060193,0.0,0.0,0.0,0.0,0.0,0.077152,0.0,0.000000,...,0.0,0.0,0.0,0.037389,0.039820,0.000000,0.038236,0.040161,0.09759,0.0
Accessory Bags,0.060193,1.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.017025,...,0.0,0.0,0.0,0.045011,0.019175,0.021979,0.000000,0.038678,0.00000,0.0
Accident Prevention Equipment,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.015264,0.016256,0.000000,0.000000,0.032791,0.00000,0.0


### 특정 상품 유사도 추출
- 'Yogurt Drinks' 상품과 유사도가 가장 높은 상위 5개 상품 추출

In [6]:
merchan_sim_df["Yogurt Drinks"].sort_values(ascending=False)[1:6]

clac_nm3
General Snacks        0.482155
Fresh Milk            0.482012
Chicken Eggs          0.479976
Spoon Type Yogurts    0.477880
Tofu                  0.470498
Name: Yogurt Drinks, dtype: float64

### 전체 상품에 대한 상품 코드 예측
- predic_trating 함수를 정의하고, ratings_pred_matrix에 데이터 프레임 형태로 저장하여 출력하세요.

In [7]:
def predict_rating(ratings_arr, merchan_sim_arr ):
    ratings_pred = ratings_arr.dot(merchan_sim_arr)/ np.array([np.abs(merchan_sim_arr).sum(axis=1)])
    return ratings_pred

In [8]:
ratings_pred = predict_rating(ratings_matrix.values , merchan_sim_df.values)
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)
ratings_pred_matrix.head(3)

clac_nm3,,Accessory Bags,Accident Prevention Equipment,Adhesive Tapes,Adhesives,Adult's Bed Covers and Skirts,Adult's Diapers,Adult's Disposable Briefes,Adults' Bed Fillings,Adults' Bedding Sets,...,Yoga / Pilates Clothing,Yoga Mats / Exercise Mats,Yoghurts / Chunggukjang Makers,Yogurt Drinks,Young Pumpkins,Young Radishes,Yuzus / Quinces,Zipper Poly Bags / Plastic Bags,kelp,life Vests / Safety Accessories
clnt_id,,,,,,,,,,,,,,,,,,,,,
2,0.012300,0.009045,0.007467,0.008229,0.010892,0.015854,0.006684,0.010192,0.009248,0.011271,...,0.012163,0.004203,0.008294,0.016102,0.015277,0.011604,0.010132,0.012060,0.010493,0.013583
9,0.052859,0.050875,0.034678,0.049847,0.050869,0.036422,0.040643,0.065125,0.034191,0.051500,...,0.033866,0.054011,0.067044,0.082844,0.086547,0.069561,0.063618,0.063606,0.075088,0.063010
12,0.002020,0.001669,0.001439,0.001986,0.001951,0.000617,0.001275,0.002437,0.001893,0.002202,...,0.002088,0.002027,0.000000,0.003153,0.003151,0.002867,0.002699,0.002555,0.002565,0.002677


### 전체 상품에 대한 상품 코드 예측 평가
- get_mse 함수를 생성하여, 전체 상품에 대한 mse를 구하세요.

In [9]:
from sklearn.metrics import mean_squared_error

def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print('아이템 기반 모든 인접 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values ))

아이템 기반 모든 인접 이웃 MSE:  0.7433403825371361


## 문제 2. 특정 상품과 가장 비슷한 유사도를 가지는 영화에 대해서만 유사도 벡터 적용 (1번과 연결)

### 1) 특정 상품과 가장 비슷한 유사도를 가지는 상품을 10개 나열하는 함수 만들기.

In [10]:
def predict_rating_topsim(ratings_arr, merchan_sim_arr, n=10):
    pred = np.zeros(ratings_arr.shape)
    
    for col in range(ratings_arr.shape[1]):
        top_n_merchans=[np.argsort(merchan_sim_arr[:col])[:-n-1:-1]]
        
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = merchan_sim_arr[col,:][top_n_merchans].dot(ratings_arr[row,:][top_n_merchans].T)
            pred[row, col] /= np.sum(np.abs(merchan_sim_arr[col,:][top_n_merchans]))
            
            
    return pred

### 2) 위 문제에서 만든 함수를 이용해 예측 구매율을 계산하고, 실제 구매율과의 MSE 구하기.

In [ ]:
ratings_pred = predict_rating_topsim(ratings_matrix.values, merchan_sim_df.values, n=10)
print('아이템 기반 최근접 TOP 이웃 MSE:',get_mse(ratings_pred, ratings_matrix.values))

ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index, columns = ratings_matrix.columns)

### 3) 고객번호 9번에 대해 상품 추천하기 (clnt_id=9)
- clnt_id =9 가 구매한 상품 상품 나열

In [16]:
user_rating_id = ratings_matrix.loc[9,:]
user_rating_id[user_rating_id>0].sort_values(ascending=False)[:10]

clac_nm3
Young Pumpkins      1
Fresh Milk          1
Kiwi                1
Glutinous Rice      1
General Snacks      1
Functional Milk     1
Frozen Hotdogs      1
Frozen Dumplings    1
Fish Cakes          1
White Wine          1
Name: 9, dtype: int64

- 고객이 구매하지 않은 상품을 리스트 객체로 반환하는 함수 생성 

In [21]:
def get_unbought_merchan(ratings_matrix, userId):
    user_rating = ratings_matrix.loc[userId,:]
    already_bought = user_rating[user_rating>0].index.tolist()
    
    merchan_list = ratings_matrix.columns.tolist()
    unbought_list=[merchan for merchan in merchan_list if merchan not in already_bought]
    
    return unbought_list

- 위에서 만든 두 함수를 이용해 최종적으로 고객에게 상품을 추천하는 함수 만들기.

In [22]:
def recomm_merchan_by_userid(pred_df, userId, unbought_list, top_n=10):
    recomm_merchan=pred_df.loc[userId, unbought_list].sort_values(ascending=False)[:top_n]
    return recomm_merchan

unbought_list=get_unbought_merchan(ratings_matrix,9)
recomm_merchan=recomm_merchan_by_userid(ratings_pred_matrix,9,unbought_list,top_n=10)

recomm_merchan=pd.DataFrame(data=recomm_merchan.values, index=recomm_merchan.index, columns=['pred_score'])

recomm_merchan

,pred_score
clac_nm3,
Kids' Beds,0.120677
Window Air Conditioners,0.112922
Green Lavers,0.105094
Bedroom Sets,0.099510
Sun Shelters,0.099497
"Chilled Chicken, Jokbal and Pork Feet",0.096800
Coconuts,0.095704
Australian Imported Beefs - Chunks,0.095700
Frozen Tteokbokkis,0.093701


### 2) 행렬분해를 이용해 잠재 요인 협업 필터링 직접 구현하기.

In [28]:
import numpy as np
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    full_pred_matrix = np.dot(P, Q.T)
    
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
      
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse


def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda = 0.01):
    num_users, num_merchan = R.shape
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_merchan, K))

    break_count = 0
       
    non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_merchan) if R[i,j] > 0 ]
   
    for step in range(steps):
        for i, j, r in non_zeros:
            eij = r - np.dot(P[i, :], Q[j, :].T)
            P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])
       
        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 10) == 0 :
            print("### iteration step : ", step," rmse : ", rmse)
            
    return P, Q

In [24]:
cs = pd.read_csv('L사_고객정보.csv')
gd = pd.read_csv('L사_상품정보.csv')
tr = pd.read_csv('L사_거래정보.csv')

gd.pd_c = gd.pd_c.astype(str).str.zfill(4) # 상품과 거래 데이터의 pd_c 컬럼의 데이터 유형이 서로 달라 맞춰줌. 자리수까지도 맞춰줘야 함. 
df = pd.merge(tr, cs).merge(gd, on='pd_c')
df.de_dt = df.de_dt.astype(str).astype('datetime64') # 날짜정보를 담고 있는 de_dt 컬럼의 데이터 유형이 정수이기 때문
ratings_matrix = pd.pivot_table(df, index='clnt_id', columns='clac_nm3', values = 'pd_c', 
                                aggfunc=lambda x: 1 if len(x) >= 1 else 0, fill_value=0)

In [29]:
P, Q = matrix_factorization(ratings_matrix.values, K=50, steps=200, learning_rate=0.01, r_lambda = 0.01)
pred_matrix = np.dot(P, Q.T)

### iteration step :  0  rmse :  0.9885364693233958
### iteration step :  10  rmse :  0.14587048643917508
### iteration step :  20  rmse :  0.09007385087905226
### iteration step :  30  rmse :  0.06698518099604414
### iteration step :  40  rmse :  0.054039882210894126
### iteration step :  50  rmse :  0.045509154323755054
### iteration step :  60  rmse :  0.0393449203621944
### iteration step :  70  rmse :  0.034639553687105376
### iteration step :  80  rmse :  0.030918265053869572
### iteration step :  90  rmse :  0.027903075558490094
### iteration step :  100  rmse :  0.025417132285831086
### iteration step :  110  rmse :  0.02334026472413604
### iteration step :  120  rmse :  0.021586827758968346
### iteration step :  130  rmse :  0.020093733972761332
### iteration step :  140  rmse :  0.018813327878717912
### iteration step :  150  rmse :  0.017708870288264747
### iteration step :  160  rmse :  0.016751617825304467
### iteration step :  170  rmse :  0.015918871265068923
### iterati

In [30]:
ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)

In [31]:
# 고객이 구매하지 않은 상품명 추출   
unbought_list = get_unbought_merchan(ratings_matrix, 9)

# 상품 기반의 인접 이웃 협업 필터링으로 영화 추천 
recomm_merchan = recomm_merchan_by_userid(ratings_pred_matrix, 9, unbought_list, top_n=10)

# 추천상품 데이타를 DataFrame으로 생성. 
recomm_merchan = pd.DataFrame(data=recomm_merchan.values,index=recomm_merchan.index,columns=['pred_score'])
recomm_merchan

,pred_score
clac_nm3,
Bookcases / File Cabinets,1.064695
Laptops,1.056898
Skin Care Devices,1.051682
Mouses,1.047507
Electric Grills,1.042284
TV Stand,1.041855
Cooking Utensil Sets,1.034948
Hair Dryers,1.034888
Men's Climbing Pants,1.034510


## 문제3) Surprise를 활용한 책 추천 시스템 구축

In [2]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
import pandas as pd
from surprise import Reader, Dataset

### 3-1) book_ratings 데이터를 불러온 후, 아래의 형식과 같이 출력해주세요.
- Total number of ratings : xxx
- Number of unique users : xxx
- Number of unique movies : xxx

In [4]:
ratings=pd.read_csv('book_ratings.csv')
ratings.head()

,user_id,book_id,rating
0,1,1180,4
1,1,4893,3
2,1,6285,4
3,2,8034,4
4,2,8855,5


In [6]:
print('Total number of ratings : ', ratings.shape[0])
print('Number of unique users: ', ratings['user_id'].nunique())
print('Number of unique books: ',ratings['book_id'].nunique())

Total number of ratings :  981756
Number of unique users:  53424
Number of unique books:  10000


### books 데이터를 불러온 후, id, authors, title에 해당하는 열만 추출하여 books에 저장해주세요

In [9]:
books=pd.read_csv("books.csv",encoding='latin1')
books=books[['id','authors','title']]
books

,id,authors,title
0,1,Suzanne Collins,"The Hunger Games (The Hunger Games, #1)"
1,2,"J.K. Rowling, Mary GrandPrÃ©",Harry Potter and the Sorcerer's Stone (Harry P...
2,3,Stephenie Meyer,"Twilight (Twilight, #1)"
3,4,Harper Lee,To Kill a Mockingbird
4,5,F. Scott Fitzgerald,The Great Gatsby
...,...,...,...
9990,9996,Ilona Andrews,"Bayou Moon (The Edge, #2)"
9991,9997,Robert A. Caro,"Means of Ascent (The Years of Lyndon Johnson, #2)"
9992,9998,Patrick O'Brian,The Mauritius Command
9993,9999,Peggy Orenstein,Cinderella Ate My Daughter: Dispatches from th...


### 3-2) Ratings 데이터프레임에서 surprise 데이터 세트로 로딩하여 RMSE를 출력해주세요
- 알고리즘은 SVD, n_factors=30
- test_size=0.25, random_state=0
- book_ratings에서 rating의 범위를 기반으로 rating_scale 설정

In [10]:
ratings.describe()

,user_id,book_id,rating
count,981756.000000,981756.000000,981756.000000
mean,25616.759933,4943.275636,3.856534
std,15228.338826,2873.207415,0.983941
min,1.000000,1.000000,1.000000
25%,12372.000000,2457.000000,3.000000
50%,25077.000000,4921.000000,4.000000
75%,38572.000000,7414.000000,5.000000
max,53424.000000,10000.000000,5.000000


In [11]:
import pandas as pd
from surprise import Reader, Dataset

reader=Reader(rating_scale=(1,5))

#ratings DataFrame에서 칼럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 합니다.
data=Dataset.load_from_df(ratings[['user_id','book_id','rating']],reader)
trainset,testset=train_test_split(data, test_size=.25,random_state=0)

algo=SVD(n_factors=30, random_state=0)
algo.fit(trainset)
predictions=algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8405


0.8405238041196965

### 3-3) GridSearchCV를 이용하여 최적 파라미터를 산출하세요
- param_grid={'n_epochs':[15,20,25],'n_factors':[50,80,100,120]}

In [7]:
from surprise.model_selection import GridSearchCV

param_grid={'n_epochs':[15,20,25],'n_factors':[50,80,100,120]}

gs=GridSearchCV(SVD, param_grid, measures=['rmse'],cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8822198826063894
{'n_epochs': 25, 'n_factors': 50}


### 3-4) DatasetAutoFolds를 활용하여 개인 영화 추천 시스템을 구축하세요

- 1)book_ratings 에서 헤드를 제거하여 새로운 파일을 생성해주세요. (book_ratings_noh.csv)

In [12]:
ratings=pd.read_csv('book_ratings.csv')
#book_ratings_noh.csv 파일로 언로드 시 인덱스와 헤더를 모두 제거한 새로운 파일 생성
ratings.to_csv('book_ratings_noh.csv',index=False, header=False)

- 2) book_ratings_noh.csv 파일 기반으로 전체 데이터를 SVD로 학습 시켜주세요. (n_epochs와 n_factors는 GridSearchCV를 기반으로 나온 최적 파라미터, random_state=0)

In [13]:
from surprise.dataset import DatasetAutoFolds

reader=Reader(line_format='user item rating',sep=',',rating_scale=(1.0,5.0))

data_folds=DatasetAutoFolds(ratings_file='book_ratings_noh.csv',reader=reader)

#전체 데이터를 학습 데이터로 생성함
trainset=data_folds.build_full_trainset()

algo=SVD(n_epochs=25, n_factors=50,random_state=0)
algo.fit(trainset)

- 교재에서 사용한 get_unseen_surprise와 recomm_movie_by_surprise 함수를 기반으로 userID=10인 사용자에게 top10 개의 책 리스트와 예측 평점을 산출하세요.

In [21]:
def get_unseen_surprise(ratings, books, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 몯느 영화를 리스트로 생성
    seen_books=ratings[ratings['user_id']==userId]['book_id'].tolist()
    
    #모든 영화의 movieId를 리스트로 생성.
    total_books=books['id'].tolist()
    
    #모든 영화의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외한 후 리스트로 생성
    unseen_books=[book for book in total_books if book not in seen_books]
    print('평점 매긴 책 수: ',len(seen_books),'추천 대상 책 수: ',len(unseen_books),
         '전체 책 수: ',len(total_books))
    
    return unseen_books

unseen_books=get_unseen_surprise(ratings, books, 10)

평점 매긴 책 수:  16 추천 대상 책 수:  9979 전체 책 수:  9995


In [22]:
def recomm_book_by_surprise(algo, userId, unseen_books, top_n=10):
    
    predictions=[algo.predict(str(userId), str(id)) for id in unseen_books]
    
    def sortkey_est(pred):
        return pred.est
    
    predictions.sort(key=sortkey_est, reverse=True)
    
    top_predictions=predictions[:top_n]
    
    #top_n으로 추출된 책의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_book_ids=[ int(pred.iid) for pred in top_predictions]
    top_book_rating=[pred.est for pred in top_predictions]
    top_book_titles=books[books.id.isin(top_book_ids)]['title']
    
    top_book_preds=[ (id, title, rating) for id, title, rating in \
                    zip(top_book_ids, top_book_titles, top_book_rating)]
    
    return top_book_preds

unseen_books=get_unseen_surprise(ratings, books, 10)
top_book_preds=recomm_book_by_surprise(algo, 10, unseen_books, top_n=10)

print('#### TOP 10 추천 책 리스트 ####')
for top_book in top_book_preds:
    print(top_book[1],":",top_book[2])

평점 매긴 책 수:  16 추천 대상 책 수:  9979 전체 책 수:  9995
#### TOP 10 추천 책 리스트 ####
A Game of Thrones (A Song of Ice and Fire, #1) : 4.891455596919947
Words of Radiance (The Stormlight Archive, #2) : 4.877056710164582
Jesus the Christ : 4.854118497561855
The Days Are Just Packed: A Calvin and Hobbes Collection : 4.849581190126291
The Authoritative Calvin and Hobbes: A Calvin and Hobbes Treasury : 4.843138740752508
The Indispensable Calvin and Hobbes : 4.8345439051958605
Homicidal Psycho Jungle Cat: A Calvin and Hobbes Collection : 4.831507374433395
The Book of Disquiet : 4.8278391038455615
Crush : 4.803944680284454
Humans of New York: Stories : 4.762878963076419
